# **Decision Trees**

Decision trees are machine learning models that try to find patterns in the features of data points. Take a look at the tree below. This tree tries to predict whether a student will get an A on their next test.

By asking questions like “What is the student’s average grade in the class” the decision tree tries to get a better understanding of their chances on the next test.

In order to make a classification, this classifier needs a data point with four features:
- The student’s average grade in the class.
- The number of hours the student plans on studying for the test.
- The number of hours the student plans on sleeping the night before the test.
- Whether or not the student plans on cheating.

For example, let’s say that somebody has a “B” average in the class, studied for more than 3 hours, slept less than 5 hours before the test, and doesn’t plan to cheat. If we start at the top of the tree and take the correct path based on that data, we’ll arrive at a leaf node that predicts the person will not get an A on the next test.

<img src="DT.png" width="50%" height="50%">

## Making Decision Trees

If we’re given this magic tree, it seems relatively easy to make classifications. But how do these trees get created in the first place? Decision trees are supervised machine learning models, which means that they’re created from a training set of labeled data. Creating the tree is where the *learning* in machine learning happens.

Take a look at the images below. We begin with every point in the training set at the top of the tree. These training points have labels — the red points represent students that didn’t get an A on a test and the green points represent students that did get an A on a test .

We then decide to split the data into smaller groups based on a feature. For example, that feature could be something like their average grade in the class. Students with an A average would go into one set, students with a B average would go into another subset, and so on.

Once we have these subsets, we repeat the process — we split the data in each subset again on a different feature.

Eventually, we reach a point where we decide to stop splitting the data into smaller groups. We’ve reached a leaf of the tree. We can now count up the labels of the data in that leaf. If an unlabeled point reaches that leaf, it will be classified as the majority label.

We can now make a tree, but how did we know which features to split the data set with? After all, if we started by splitting the data based on the number of hours they slept the night before the test, we’d end up with a very different tree that would produce very different results. How do we know which tree is best?

<img src="DTa.png" width="30%" height="30%">  <img src="DTb.png" width="30%" height="30%">  <img src="DTc.png" width="30%" height="30%">
<img src="DTd.png" width="30%" height="30%">  <img src="DTe.png" width="30%" height="30%">  <img src="DTf.png" width="30%" height="30%">
<img src="DTg.png" width="30%" height="30%">  <img src="DTh.png" width="30%" height="30%">

## Data

We'll be working with a dataset about drug consumption. When considering the risk of a person consuming a particular drug, what are the biggest factors?

Drug use fell into one of the following categories `never used`, `used over a decade ago`, `used in the last decade`, `used in the last year`, `used in the last month`, `used in the last week`, and `used in the last day`.

The following attributes were also collected:
- age
- gender
- education
- country of current residence
- ethnicity
- neuroticism score
- extraversion score
- openness to experience score
- agreeableness score
- conscientiousness score
- impulsiveness score
- sensation seeing score

In [1]:
from collections import Counter
import numpy as np
import operator
import pandas as pd
from random import sample

## Gini Impurity

Consider the two trees below. Which tree would be more useful as a model that tries to predict whether someone would get an A in a class?

<img src="ginia.png" width="40%" height="40%">  <img src="ginib.png" width="40%" height="40%">

Let’s say you use the top tree. You’ll end up at a leaf node where the label is up for debate. The training data has labels from both classes! If you use the bottom tree, you’ll end up at a leaf where there’s only one type of label. There’s no debate at all! We’d be much more confident about our classification if we used the bottom tree.

This idea can be quantified by calculating the *Gini impurity* of a set of data points. To find the Gini impurity, start at `1` and subtract the squared percentage of each label in the set. For example, if a data set had three items of class `A` and one item of class `B`, the Gini impurity of the set would be

$$1 - \bigg(\frac{3}{4}\bigg)^2 = 0.375$$

If a data set has only one class, you’d end up with a Gini impurity of `0`. The lower the impurity, the better the decision tree!

In [2]:
drugs = pd.read_csv("drug_consumption.data", header=None)
labels = list(drugs[28]) # mushrooms

# create a random sample of labels from the nursery dataset
labels_sub = sample(labels, 10)

# count the number of times each unique label is in the dataset
label_counts = Counter(labels_sub)
print(label_counts)

# derive the impurity of each label
impurity = 1
for label in label_counts:
    label_prob = label_counts[label] / len(labels_sub)
    impurity = impurity - label_prob ** 2
    print(impurity)

Counter({'CL0': 6, 'CL3': 2, 'CL4': 1, 'CL2': 1})
0.64
0.63
0.62
0.58


## Information Gain

We know that we want to end up with leaves with a low Gini Impurity, but we still need to figure out which features to split on in order to achieve this. For example, is it better if we split our dataset of students based on how much sleep they got or how much time they spent studying?

To answer this question, we can calculate the information gain of splitting the data on a certain feature. Information gain measures difference in the impurity of the data before and after the split. For example, let’s say you had a dataset with an impurity of `0.5`. After splitting the data based on a feature, you end up with three groups with impurities `0`, `0.375`, and `0`. The information gain of splitting the data in that way is `0.5 - 0 - 0.375 - 0 = 0.125`.

Not bad! By splitting the data in that way, we’ve gained some information about how the data is structured — the datasets after the split are purer than they were before the split. The higher the information gain the better — if information gain is `0`, then splitting the data on that feature was useless! Unfortunately, right now it’s possible for information gain to be negative. Next, we’ll calculate *weighted* information gain to fix that problem.

<img src="imf_gain.png" width="40%" height="40%">

In [3]:
# create another random sample of labels
labels_sub = sample(labels, 15)

# first list of lists
split_1 = [arr.tolist() for arr in np.array_split(labels_sub, 2)]

# first sublist
split2a = np.random.choice(labels_sub, 7, replace=False).tolist()
# second sublist
diff = Counter(labels_sub) - Counter(split2a)
remainder = list(diff.elements())
split2b = np.random.choice(remainder, 4, replace=False).tolist()
# final sublist
diff = Counter(remainder) - Counter(split2b)
split2c = list(diff.elements())
# combine into a list of lists
split_2 = [split2a, split2b, split2c]


# derive the gini impurity
def gini(data):
    impurity = 1
    label_counts = Counter(data)
    
    for label in label_counts:
        label_prob = label_counts[label] / len(data)
        impurity -= label_prob ** 2
    return impurity


# initialise the information gain as the gini impurity of the unsplit list
info_gain = gini(labels_sub)
# calculate the information gain on each sublist
for sublist in split_1:
    info_gain -= gini(sublist)
print(info_gain)


# as a function
def information_gain(starting_labels, split_labels):
    info_gain = gini(starting_labels)
    for sublist in split_labels:
        info_gain -= gini(sublist)
    return info_gain

-0.5791893424036282


## Weighted Information Gain

We’re not quite done calculating the information gain of a set of objects. The sizes of the subset that get created after the split are important too! For example, the image below shows two sets with the same impurity. Which set would you rather have in your decision tree?

<img src="winf_gaina.png" width="40%" height="40%">

Both of these sets are perfectly pure, but the purity of the second set is much more meaningful. Because there are so many items in the second set, we can be confident that whatever we did to produce this set wasn’t an accident.

It might be helpful to think about the inverse as well. Consider these two sets with the same impurity:

<img src="winf_gainb.png" width="40%" height="40%">

Both of these sets are completely impure. However, that impurity is much more meaningful in the set with more instances. We know that we are going to have to do a lot more work in order to completely separate the two classes. Meanwhile, the impurity of the set with two items isn’t as important. We know that we’ll only need to split the set one more time in order to make two pure sets.

Let’s modify the formula for information gain to reflect the fact that the size of the set is relevant. Instead of simply subtracting the impurity of each set, we’ll subtract the *weighted* impurity of each of the split sets. If the data before the split contained `20` items and one of the resulting splits contained `2` items, then the weighted impurity of that subset would be `2/20 * impurity`. We’re lowering the importance of the impurity of sets with few elements.

<img src="winf_gainc.png" width="40%" height="40%">

Now that we can calculate the information gain using weighted impurity, let’s do that for every possible feature. If we do this, we can find the best feature to split the data on.

In [4]:
with open("drug_consumption.data", "r") as f:
    attributes = []
    drug = []
    for line in f:
        attributes.append(line.strip().split(",")[1:13])
        drug.append(line.strip().split(",")[28])

In [5]:
# update the information gain function to weighted information gain
def information_gain(starting_labels, split_labels):
    info_gain = gini(starting_labels)
    for sublist in split_labels:
        gini_sublist = gini(sublist) * len(sublist) / len(starting_labels)
        info_gain -= gini_sublist
    return info_gain


# split_data contains the features split into different subsets
# split_labels contains the labels of those features split into different subsets
def split(features, labels, column):
    data_subsets = []
    label_subsets = []
    counts = list(set([data[column] for data in features]))
    counts.sort()
    for k in counts:
        new_data_subset = []
        new_label_subset = []
        for i in range(len(features)):
            if features[i][column] == k:
                new_data_subset.append(features[i])
                new_label_subset.append(labels[i])
        data_subsets.append(new_data_subset)
        label_subsets.append(new_label_subset)
    return data_subsets, label_subsets

# split the data based on the third index
split_data, split_labels = split(attributes, drug, 3)
# calculate the information gain
information_gain(drug, split_labels)

0.06226489486491088

In [6]:
# loop through the features and calculate the information gain for each
for i, feat in enumerate(attributes[0]):
    print(i)
    split_data, split_labels = split(attributes, drug, i)
    print(information_gain(drug, split_labels))

0
0.034061673932833984
1
0.020844654632403647
2
0.025418215110499666
3
0.06226489486491088
4
0.00650163611199013
5
0.017292636249597558
6
0.016781178663765408
7
0.04587936977331512
8
0.01837607600378823
9
0.032280445818495526
10
0.028571207643427285
11
0.045543110242563786


## Recursive Tree Building

Now that we can find the best feature to split the dataset, we can repeat this process again and again to create the full tree. This is a recursive algorithm! We start with every data point from the training set, find the best feature to split the data, split the data based on that feature, and then recursively repeat the process again on each subset that was created from the split.

We’ll stop the recursion when we can no longer find a feature that results in any information gain. In other words, we want to create a leaf of the tree when we can’t find a way to split the data that makes purer subsets.

The leaf should keep track of the classes of the data points from the training set that ended up in the leaf. In our implementation, we’ll use a `Counter` object to keep track of the counts of labels.

We’ll use these counts to make predictions about new data that we give the tree.

In [7]:
# function to determine the index of the feature that causes the best split and the information gain
def find_best_split(dataset, labels):
    best_gain = 0
    best_feature = 0
    for feature in range(len(dataset[0])):
        data_subsets, label_subsets = split(dataset, labels, feature)
        gain = information_gain(labels, label_subsets)
        if gain > best_gain:
            best_gain, best_feature = gain, feature
    return best_feature, best_gain

best_feature, best_gain = find_best_split(attributes, drug)
print(best_feature) # education
print(best_gain)

3
0.06226489486491088


In [8]:
def build_tree(data, labels):
    best_feature, best_gain = find_best_split(data, labels)
    if best_gain == 0:
        return Counter(labels)
    data_subsets, labels_subsets = split(data, labels, best_feature)

    branches = []
    for i in range(len(data_subsets)):
        branches.append(build_tree(data_subsets[i], labels_subsets[i]))
    return branches


def print_tree(node, spacing = ""):
    feature_dict = {0: "Age",
                    1: "gender",
                    2: "Education",
                    3: "Country of residence",
                    4: "Ethnicity",
                    5: "Neuroticism score",
                    6: "Extraversion score",
                    7: "Openness to experience score",
                    8: "Agreeableness score",
                    9: "Conscientiousness score",
                    10: "Impulsiveness score",
                    11: "Sensation seeing score"}
    # base case
    if isinstance(node, Counter):
        print(spacing + str(node))
        return
    # print the feature at this node
    print(spacing + "Splitting")
    
    # call the function recursively on the true branch
    for i in range(len(node)):
        print(spacing + '--> Branch ' + str(i) + ':')
        print_tree(node[i], spacing + " ")

tree = build_tree(attributes, drug)
print_tree(tree)

Splitting
--> Branch 0:
 Splitting
 --> Branch 0:
  Counter({'CL3': 1})
 --> Branch 1:
  Counter({'CL3': 2})
 --> Branch 2:
  Splitting
  --> Branch 0:
   Counter({'CL4': 1})
  --> Branch 1:
   Counter({'CL0': 1})
 --> Branch 3:
  Splitting
  --> Branch 0:
   Counter({'CL4': 1})
  --> Branch 1:
   Counter({'CL3': 1})
  --> Branch 2:
   Counter({'CL4': 1})
 --> Branch 4:
  Splitting
  --> Branch 0:
   Counter({'CL6': 1})
  --> Branch 1:
   Counter({'CL0': 1})
  --> Branch 2:
   Counter({'CL2': 1})
 --> Branch 5:
  Splitting
  --> Branch 0:
   Counter({'CL0': 1})
  --> Branch 1:
   Counter({'CL1': 1})
  --> Branch 2:
   Counter({'CL5': 1})
 --> Branch 6:
  Counter({'CL0': 3})
 --> Branch 7:
  Counter({'CL0': 1})
 --> Branch 8:
  Counter({'CL0': 2})
 --> Branch 9:
  Splitting
  --> Branch 0:
   Counter({'CL3': 1})
  --> Branch 1:
   Counter({'CL0': 1})
  --> Branch 2:
   Counter({'CL2': 1})
 --> Branch 10:
  Counter({'CL0': 1})
 --> Branch 11:
  Counter({'CL3': 1})
 --> Branch 12:
  Split

## Classifying New Data

Given a new data point, we start at the top of the tree and follow the path of the tree until we hit a leaf. Once we get to a leaf, we’ll use the classes of the points from the training set to make a classification.

We’ve slightly changed the way our `build_tree()` function works. Instead of returning a list of branches or a `Counter` object, the `build_tree()` function now returns a `Leaf` object or an `Internal_Node` object.

In [9]:
# define leaf and decision node classes
class Leaf:
    def __init__(self, labels, value):
        self.labels = Counter(labels)
        self.value = value

        
class Internal_Node:
    def __init__(self, feature, branches, value):
        self.feature = feature
        self.branches = branches
        self.value = value


def build_tree(data, labels, value = ""):
    best_feature, best_gain = find_best_split(data, labels)
    if best_gain == 0:
        return Leaf(Counter(labels), value)
    data_subsets, labels_subsets = split(data, labels, best_feature)

    branches = []
    for i in range(len(data_subsets)):
        branch = build_tree(data_subsets[i], labels_subsets[i], data_subsets[i][0][best_feature])
        branches.append(branch)
    return Internal_Node(best_feature, branches, value)


def print_tree(node, spacing = ""):
    feature_dict = {0: "Age",
                    1: "gender",
                    2: "Education",
                    3: "Country of residence",
                    4: "Ethnicity",
                    5: "Neuroticism score",
                    6: "Extraversion score",
                    7: "Openness to experience score",
                    8: "Agreeableness score",
                    9: "Conscientiousness score",
                    10: "Impulsiveness score",
                    11: "Sensation seeing score"}
    # base case
    if isinstance(node, Leaf):
        print(spacing + str(node.labels))
        return
    # print the feature at this node
    print(spacing + "Splitting on " + feature_dict[node.feature])
    
    # call the function recursively on the true branch
    for i in range(len(node.branches)):
        print(spacing + '--> Branch ' + node.branches[i].value + ':')
        print_tree(node.branches[i], spacing + " ")

tree = build_tree(attributes, drug)
print_tree(tree)

Splitting on Country of residence
--> Branch -0.09765:
 Splitting on Neuroticism score
 --> Branch -0.05188:
  Counter({'CL3': 1})
 --> Branch -0.14882:
  Counter({'CL3': 2})
 --> Branch -0.24649:
  Splitting on Age
  --> Branch -0.95197:
   Counter({'CL4': 1})
  --> Branch 0.49788:
   Counter({'CL0': 1})
 --> Branch -0.34799:
  Splitting on Extraversion score
  --> Branch 0.16767:
   Counter({'CL4': 1})
  --> Branch 0.32197:
   Counter({'CL3': 1})
  --> Branch 0.80523:
   Counter({'CL4': 1})
 --> Branch -0.58016:
  Splitting on Extraversion score
  --> Branch 0.32197:
   Counter({'CL6': 1})
  --> Branch 0.47617:
   Counter({'CL0': 1})
  --> Branch 0.63779:
   Counter({'CL2': 1})
 --> Branch -0.67825:
  Splitting on Extraversion score
  --> Branch 0.32197:
   Counter({'CL0': 1})
  --> Branch 0.63779:
   Counter({'CL1': 1})
  --> Branch 1.74091:
   Counter({'CL5': 1})
 --> Branch -0.92104:
  Counter({'CL0': 3})
 --> Branch -1.05308:
  Counter({'CL0': 1})
 --> Branch -1.19430:
  Counter(

In [10]:
def classify(datapoint, tree):
    # check if we're at a leaf
    if isinstance(tree, Leaf):
        # get the label with the highest count
        return max(tree.labels.items(), key=operator.itemgetter(1))[0]
    # otherwise find the branch corresponding to the datapoint
    value = datapoint[tree.feature]

    for branch in tree.branches:
        if branch.value == value:
            return classify(datapoint, branch)

In [11]:
# choose a random application to test
test_point = sample(attributes, 1)[0]
print(test_point)

pred = classify(test_point, tree)
print(pred)

['0.49788', '-0.48246', '0.45468', '0.96082', '-0.31685', '-0.79151', '-0.30033', '-0.17779', '1.11406', '-0.52745', '-0.71126', '0.07987']
CL2


## Decision Trees in scikit-learn

Nice work! You’ve written a decision tree from scratch that is able to classify new points. Let’s take a look at how the Python library `scikit-learn` implements decision trees.

The `sklearn.tree` module contains the `DecisionTreeClassifier` class. To create a `DecisionTreeClassifier` object, call the constructor:
```
classifier = DecisionTreeClassifier()
```
Next, we want to create the tree based on our training data. To do this, we’ll use the `.fit()` method.

`.fit()` takes a list of data points followed by a list of the labels associated with that data. Note that when we built our tree from scratch, our data points contained strings like `"vhigh"` or `"5more"`. When creating the tree using `scikit-learn`, it’s a good idea to map those strings to numbers. For example, for the first feature representing the price of the car, `"low"` would map to `1`, `"med"` would map to `2`, and so on.
```
classifier.fit(training_data, training_labels)
```
Finally, once we’ve made our tree, we can use it to classify new data points. The `.predict()` method takes an array of data points and will return an array of classifications for those data points.
```
predictions = classifier.predict(test_data)
```
If you’ve split your data into a test set, you can find the accuracy of the model by calling the `.score()` method using the test data and the test labels as parameters.
```
print(classifier.score(test_data, test_labels))
```
`.score()` returns the percentage of data points from the test set that it classified correctly.

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

In [13]:
drugs = pd.read_csv("drug_consumption.data", header=None)
# filter out unnecessary columns
drugs.drop(0, inplace=True, axis=1)
drugs.drop(drugs.iloc[:, 12:27], inplace=True, axis=1)
drugs.drop(drugs.iloc[:, -3:], inplace=True, axis=1)
drugs.head()

,1,2,3,4,5,6,7,8,9,10,11,12,28
0,0.49788,0.48246,-0.05921,0.96082,0.12600,0.31287,-0.57545,-0.58331,-0.91699,-0.00665,-0.21712,-1.18084,CL0
1,-0.07854,-0.48246,1.98437,0.96082,-0.31685,-0.67825,1.93886,1.43533,0.76096,-0.14277,-0.71126,-0.21575,CL0
2,0.49788,-0.48246,-0.05921,0.96082,-0.31685,-0.46725,0.80523,-0.84732,-1.62090,-1.01450,-1.37983,0.40148,CL1
3,-0.95197,0.48246,1.16365,0.96082,-0.31685,-0.14882,-0.80615,-0.01928,0.59042,0.58489,-1.37983,-1.18084,CL0
4,0.49788,0.48246,1.98437,0.96082,-0.31685,0.73545,-1.63340,-0.45174,-0.30172,1.30612,-0.21712,-0.21575,CL2


In [14]:
# split the data into 80% train and 20% validation
y = drugs[28]
X = drugs.drop(28, axis=1)

# ensure the data is shuffled and that the training and test sets are representative of the target distribution
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, stratify=y)
# view the first application in the training set
print(X_train.iloc[0, :])
print('-----')
print(y_train.iloc[0])

1     1.09449
2     0.48246
3     1.16365
4     0.96082
5    -0.31685
6    -0.34799
7    -0.80615
8    -0.45174
9    -1.21213
10   -0.14277
11    1.86203
12    0.40148
Name: 1256, dtype: float64
-----
CL0


In [15]:
clf = DecisionTreeClassifier()

clf.fit(X_train, y_train)
# assess the model's accuracy on the validation data
print(clf.score(X_test, y_test))

0.47214854111405835


## Decision Tree Limitations

Now that we have an understanding of how decision trees are created and used, let’s talk about some of their limitations.

One problem with the way we’re currently making our decision trees is that our trees aren’t always `globablly optimal`. This means that there might be a better tree out there somewhere that produces better results. But wait, why did we go through all that work of finding information gain if it’s not producing the best possible tree?

Our current strategy of creating trees is `greedy`. We assume that the best way to create a tree is to find the feature that will result in the largest information gain `right now` and split on that feature. We never consider the ramifications of that split further down the tree. It’s possible that if we split on a suboptimal feature right now, we would find even better splits later on. Unfortunately, finding a globally optimal tree is an extremely difficult task, and finding a tree using our greedy approach is a reasonable substitute.

Another problem with our trees is that they potentially `overfit` the data. This means that the structure of the tree is too dependent on the training data and doesn’t accurately represent the way the data in the real world looks like. In general, larger trees tend to overfit the data more. As the tree gets bigger, it becomes more tuned to the training data and it loses a more generalized understanding of the real world data.

One way to solve this problem is to `prune` the tree. The goal of pruning is to shrink the size of the tree. There are a few different pruning strategies, and we won’t go into the details of them here. `scikit-learn` currently doesn’t prune the tree by default, however we can dig into the code a bit to prune it ourselves.

In [16]:
# determine how big the tree is
print(clf.tree_.max_depth)

22


In [17]:
# restrict the tree's depth to 10 and reassess model accuracy
clf = DecisionTreeClassifier(max_depth=10)
clf.fit(X_train, y_train)

print(clf.score(X_test, y_test))

0.53315649867374


## Review

You learned how to create decision trees and use them to make classifications. Here are some of the major takeaways:
- Good decision trees have pure leaves. A leaf is pure if all of the data points in that class have the same label.
- Decision trees are created using a greedy algorithm that prioritizes finding the feature that results in the largest information gain when splitting the data using that feature.
- Creating an optimal decision tree is difficult. The greedy algorithm doesn’t always find the globally optimal tree.
- Decision trees often suffer from overfitting. Making the tree small by pruning helps to generalize the tree so it is more accurate on data in the real world.